# 基于FlaskRuntime服务化组件

## 目标
使用 FlaskRuntime 对组件进行服务化。

FlaskRuntime 是对组件（Component）的服务化封装，具体有如下几个功能：
- 一键服务化组件: 使得组件能够以服务的形式运行，支持 API 调用。
- Session 数据管理: 提供 Session 数据的管理功能，允许跟踪和存储用户会话数据。
- 请求时鉴权: 支持在请求时进行认证，确保安全性。


## 准备工作
### 安装Python SDK

appbuilder 支持使用 pip 安装（要求Python >= 3.8），并且 FlaskRuntime 服务化组件依赖 `appbuilder-sdk[serve]`

In [ ]:
pip install appbuilder-sdk 'appbuilder-sdk[serve]'

## 基本用法

### 快速开始

下面的示例会基于 Playground 组件，在 8091 端口部署 Web 服务: 

In [ ]:
import os
import appbuilder
from appbuilder.utils.flask_deploy import FlaskRuntime

# 使用组件之前，请前往千帆AppBuilder官网创建密钥，流程详见：https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1、创建密钥
os.environ["APPBUILDER_TOKEN"] = '...'

component = appbuilder.Playground(
    prompt_template="{query}",
    model="ERNIE-Bot"
)

agent = FlaskRuntime(component=component)
agent.serve(port=8091)

通过 Shell 命令测试启动的服务, 请求 Body 为组件 run 方法的入参：


In [ ]:
curl --location 'http://0.0.0.0:8091/chat' \
--header 'Content-Type: application/json' \
--data '{
    "message": "海淀区的面积是多少",
    "stream": false
}'

## FlaskRuntime 参数说明

### 1. 类初始化参数说明

FlaskRuntime 初始化接受两个参数。

| 参数名称 | 参数类型 | 是否必须 | 描述 | 示例值 |
|--|--|--|--|--|
| component | Component | 是 | 可运行的 Component, 该 Component 需要实现 run(message, stream, **args) 方法。 | Playground(prompt_template="{query}", model="ERNIE-Bot") |
| user_session_config | sqlalchemy.engine.URL\|Str\|None | 否 | 会话 Session 数据存储的数据库配置，遵循 sqlalchemy 后端定义，可参考[文档](https://docs.sqlalchemy.org/en/20/core/engines.html#backend-specific-urls)。默认使用 sqlite:///user_session.db，即本地的 SQLite 存储 | "sqlite:///user_session.db" |
| user_session| UserSession | 否 | 用户会话管理器，如果不指定则自动生成一个默认的 UserSession | UserSession(user_session_config) |

### 2. API 服务参数

#### 2.1 请求参数

**接口定义**

| URL | Method |
|--|--|
| /chat | POST |

**Header 参数**

| 参数名称 | 是否必须 | 描述 | 示例值 |
|--|--|--|--|
| Content-Type | 是 | 必须设置为"application/json" | "application/json" |
| X-Appbuilder-Token | 否 | 开启请求时认证能力时需要带入 APPBUILDER_TOKEN 进行鉴权 | 前往千帆AppBuilder官网创建密钥，流程详见[文档](https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1、创建密钥) |

**Body 参数**

| 参数名称 | 参数类型 | 是否必须 | 描述 | 示例值 |
|--|--|--|--|--|
| message | Any | 是 | 透传到 component 的 run 方法的 message 参数 | "海淀区的面积是多少" |
| stream | Bool | 否 | 是否流式调用。透传到 component 的 run 方法的 stream 参数。默认为 false | false |
| session_id | Str | 否 | 用于标示同一个会话（Session）。如果不传该值，后端会自动生成 session_id，在响应参数中返回 | "99680089-5acb-4298-9ade-a1a3f6c28102" |
| 其他参数 | Any | 否 | 透传到 component 的 run 方法 | - |


#### 2.2 响应参数
分为非流式响应和流式响应。

**非流式响应**

| 参数名称 | 参数类型 | 描述 | 示例值 |
|--|--|--|--|
| code | Int | 错误码。值为0表示成功，否则为失败。非0错误详见错误码部分描述 | 0 |
| message | Str | 错误信息描述。 | "Missing input variable query in message ['海淀区的面积是多少']" |
| result | Object | 请求结果 | - |
| + answer_message | Object | 组件返回值，由返回的 Message 序列化得到 | {"content":"海淀区是北京市的一个区，位于北京市主城区西部和西北部，东与西城区、朝阳区相邻，南与丰台区毗连，西与石景山区、门头沟区交界，北与昌平区接壤。海淀区的面积为**431平方千米**，约占北京市总面积的2.6%。","extra":{},"id":"6b4e5019-a708-4bc5-a6ec-595fb4285677","mtype":"dict","name":"msg"} |
| + session_id | Str | 用于标示同一个会话（Session） | "99680089-5acb-4298-9ade-a1a3f6c28102" |

**流式响应**

流式数据以追加的形式返回。流式和非流式的数据结构一致，不再描述。

#### 2.3 响应示例

分为非流式响应和流式响应。

**非流式响应**

```shell
{
  "code": 0,
  "message": "",
  "result": {
    "answer_message": {
      "content": "海淀区是北京市的一个区，位于北京市主城区西部和西北部，东与西城区、朝阳区相邻，南与丰台区毗连，西与石景山区、门头沟区交界，北与昌平区接壤。海淀区的面积为**431平方千米**，约占北京市总面积的2.6%。",
      "extra": {},
      "id": "6b4e5019-a708-4bc5-a6ec-595fb4285677",
      "mtype": "dict",
      "name": "msg"
    },
    "session_id": "99680089-5acb-4298-9ade-a1a3f6c28102"
  }
}
```

**流式响应**

```shell
data: {"code": 0, "message": "", "result": {"session_id": "663303a9-d83d-481f-a084-872ece87989c", "answer_message": {"content": "海淀区", "extra": {}}}}

data: {"code": 0, "message": "", "result": {"session_id": "663303a9-d83d-481f-a084-872ece87989c", "answer_message": {"content": "，隶属于北京市，位于北京市主城区西部和西北部，东与西城区、朝阳区相邻，南与丰台区毗连，", "extra": {}}}}

data: {"code": 0, "message": "", "result": {"session_id": "663303a9-d83d-481f-a084-872ece87989c", "answer_message": {"content": "西与石景山区、门头沟区交界，北与昌平区接壤，总面积**431平方千米**。", "extra": {}}}}

data: {"code": 0, "message": "", "result": {"session_id": "663303a9-d83d-481f-a084-872ece87989c", "answer_message": {"content": "", "extra": {}}}}
```

#### 2.4 错误码
| 错误码 | 描述 |
|--|--|
| 400 | 客户端请求参数错误 |
| 1000 | 服务端执行错误 |

## 高级用法

### 1. 一键服务化组件 
FlaskRuntime 可以快速组件以服务的形式运行，支持 API 调用和对话框交互。

**1.1 API调用**

API 调用的基础用法在快速开始小结已经给出，这里不再赘述。

下面介绍使用 `gunicorn` 启动生产级 Web 服务的方法，`gunicorn` 是一个适用于 UNIX 的 Python WSGI HTTP 服务器，详见[项目链接](https://github.com/benoitc/gunicorn)。

首先创建 `app.py` 文件，暴露 Flask App：

In [ ]:
import os
import appbuilder
from appbuilder.utils.flask_deploy import FlaskRuntime

# 使用组件之前，请前往千帆AppBuilder官网创建密钥，流程详见：https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1、创建密钥
os.environ["APPBUILDER_TOKEN"] = '...'

def get_flask_app():
    component = appbuilder.Playground(
        prompt_template="{query}",
        model="ERNIE-Bot"
    )
    agent = FlaskRuntime(component=component)
    return agent.create_flask_app()

基于 `gunicorn` 启动生产级服务：

In [ ]:
# 服务工作进程数
SERVER_WORKER_AMOUNT=8
# 服务工作进程启动方式
SERVER_WORKER_CLASS=gevent
# 服务超时时间
GUNICORN_TIMEOUT=60

gunicorn \
  --bind "0.0.0.0:8091" \
  --workers ${SERVER_WORKER_AMOUNT} \
  --worker-class ${SERVER_WORKER_CLASS} \
  --timeout ${GUNICORN_TIMEOUT} \
  "app:get_flask_app()"

### 2. Session 数据管理
请参考：[user_session.ipynb](./user_session.ipynb)

### 3. 请求时鉴权
FlaskRuntime 支持在请求时进行认证，确保安全性。

使用该能力，在初始化组件时需要设置 lazy 鉴权：

In [ ]:
import appbuilder
from appbuilder.utils.flask_deploy import FlaskRuntime

# 无需配置 APPBUILDER_TOKEN 环境变量

component = appbuilder.Playground(
    prompt_template="{query}",
    model="ERNIE-Bot",
    lazy_certification=True, # 设置 lazy 鉴权，在创建时不进行认证
)

agent = FlaskRuntime(component=component)
agent.serve(port=8091)

当初始化组件时进行了 lazy 鉴权，请求时请求头必须带上 `X-Appbuilder-Token` （即Appbuilder密钥，获取流程详见[千帆AppBuilder官网创建密钥](https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1、创建密钥)）进行鉴权：

In [ ]:
curl --location 'http://0.0.0.0:8091/chat' \
    --header 'Content-Type: application/json' \
    --header 'X-Appbuilder-Token: ...' \
    --data '{
        "message": "海淀区的面积是多少",
        "stream": false
    }'